# Libarary

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
import pandas as pd

In [10]:
from sklearn.linear_model import LogisticRegression

In [12]:
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

In [45]:
from sklearn import metrics

In [ ]:
from imblearn.over_sampling import SMOTE

# Read data

In [2]:
df = pd.read_csv('/home/ubuntu/Zabbix/Data/MODEL_DATA/Anomaly_Data/SRV_FR_SQLN_11/SRV_FR_SQLN_11_ANOMALY_COL_NAME.csv')

In [3]:
df.shape

(45270, 141)

In [4]:
df.head(1)

,Date_Time,Total disk space on K:,Total disk space on J:,Total disk space on N:\LOGS,Total disk space on N:,"SQL Server, Locks: Lock Waits per second (Total)","SQL Server, Buffer Manager: Page reads/sec",Outgoing network traffic on vmxnet3 Ethernet Adapter-WFP Native MAC Layer LightWeight Filter-0000,"SQL Server, Locks: Number of Deadlocks/sec (Total)",Free disk space on V:,...,Free disk space on H:\LOGS,Free disk space on Q:,"SQL Server, Memory Manager: Memory Grants Pending",Free disk space on S:,"SQL Server, Cache: Cache Hit Ratio",Anomaly,Anomaly_Score,Anomaly_Feature_1_1,Anomaly_Feature_2_2,Anomaly_Feature_3_3
0,2020-06-16 05:17:00,8.938692e+11,9.663355e+10,5.365498e+09,2.899071e+11,0.0,0.0,1914432.0,0.0,1.661783e+10,...,4.689887e+09,3.090481e+10,0.0,1.396507e+10,81.8883,1,0.120815,SQL Server: Number of Blocked Processes,Free disk space on J:\LOGS,Free disk space on J:


In [5]:
# Filtering only the anomaly data from all data
anomaly_df = df[df['Anomaly'] == -1]

In [14]:
anomaly_df.shape

(252, 141)

In [8]:
# Classification1
## Classification model for anomaly_feature_1_1

In [27]:
# df_1 for only classifiaction of feature_1_1
feat_1_col = [x for x in anomaly_df.columns if x not in ['Date_Time','Anomaly' ,'Anomaly_Score', 'Anomaly_Feature_2_2','Anomaly_Feature_3_3']]
df_1 = anomaly_df[feat_1_col]


In [28]:
df_1.shape

(252, 136)

In [29]:
X = df_1.loc[:, df_1.columns != 'Anomaly_Feature_1_1']
y = df_1.loc[:, df_1.columns == 'Anomaly_Feature_1_1']

In [30]:

os = SMOTE(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
columns = X_train.columns

In [31]:
X_train.columns[-1]

'SQL Server, Cache: Cache Hit Ratio'

In [40]:
y_train['Anomaly_Feature_1_1'].value_counts()

Free disk space on H:                           88
Free disk space on N:                           34
Free disk space on D:                           21
Free disk space on M:                           16
SQL Server, Buffer Manager: Lazy writes/sec      3
SQL Server: Logins/sec                           3
Total disk space on K:                           2
Total disk space on P:                           2
SQL Server, Memory Manager: Granted Workspac     1
SQL Server, Buffer Manager: % Buffer cache h     1
SQL Server: % Processor Time                     1
Processor load (15 min average)                  1
SQL Server, Latches: Total Latch Wait Time       1
SQL Server, Locks: Number of Deadlocks/sec (     1
SQL Server, Memory Manager: Memory Grants Ou     1
Name: Anomaly_Feature_1_1, dtype: int64

In [41]:
# os_data_X,os_data_y=os.fit_resample(X_train, y_train)

In [43]:
# os_data_X = pd.DataFrame(data=os_data_X,columns=columns )
# os_data_y= pd.DataFrame(data=os_data_y,columns=['Anomaly_Feature_1_1'])

In [44]:
# Model 

In [48]:
print('X train',X_train.shape)
print('y train', y_train.shape)


X train (176, 135)
y train (176, 1)


In [50]:
# Data preprocessing
from sklearn.preprocessing import StandardScaler

In [51]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [52]:
logreg1 = LogisticRegression()

In [53]:
logreg1.fit(X_train_scaled, y_train)

/home/ubuntu/anaconda3/envs/zabbix/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/home/ubuntu/anaconda3/envs/zabbix/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [54]:

# scaling X_test
X_test_scaled = scaler.transform(X_test)

In [99]:
y_pred = logreg1.predict(X_test_scaled)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg1.score(X_test_scaled, y_test)))

Accuracy of logistic regression classifier on test set: 0.50


In [108]:
print('Accuracy of model on test set: {:.2f}'.format(metrics.accuracy_score(y_pred,y_test)))

Accuracy of model on test set: 0.50


In [98]:
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg1.score(y_test, y_pred)))

ValueError: could not convert string to float: 'Free disk space on H:'

In [60]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

[[ 8  5  0  1  0  0  0  0  0  0  0]
 [ 4 24  2  8  0  0  0  0  0  0  0]
 [ 0  3  2  0  0  0  0  0  0  0  0]
 [ 3  4  1  3  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0  0  1  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  1  0  0  0  0  0]
 [ 0  1  0  0  0  0  0  0  0  0  1]
 [ 1  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  1  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  1]]


In [61]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

                                              precision    recall  f1-score   support

                       Free disk space on D:       0.47      0.57      0.52        14
                       Free disk space on H:       0.65      0.63      0.64        38
                       Free disk space on M:       0.33      0.40      0.36         5
                       Free disk space on N:       0.25      0.27      0.26        11
             Processor load (15 min average)       0.00      0.00      0.00         2
SQL Server, Buffer Manager: % Buffer cache h       0.00      0.00      0.00         0
  SQL Server, Buffer Manager: Page reads/sec       0.00      0.00      0.00         1
SQL Server, Errors: Errors/sec (Kill Connect       0.00      0.00      0.00         2
  SQL Server, Latches: Total Latch Wait Time       0.00      0.00      0.00         1
SQL Server, Memory Manager: Granted Workspac       0.00      0.00      0.00         1
                      SQL Server: Logins/sec       0.

/home/ubuntu/anaconda3/envs/zabbix/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ubuntu/anaconda3/envs/zabbix/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [65]:
df['Date_Time']

0        2020-06-16 05:17:00
1        2020-06-16 05:18:00
2        2020-06-16 05:19:00
3        2020-06-16 05:20:00
4        2020-06-16 05:21:00
                ...         
45265    2020-07-20 01:09:00
45266    2020-07-20 01:10:00
45267    2020-07-20 01:11:00
45268    2020-07-20 01:12:00
45269    2020-07-20 01:13:00
Name: Date_Time, Length: 45270, dtype: object

In [70]:
date_col = pd.Series(pd.date_range(start='2020-06-01 00:00:00', end= '2020-12-31 23:59:59', freq='1T'))

In [73]:
date_col.shape

(308160,)

In [75]:
df_year = pd.DataFrame(data = date_col)#, columns = 'Y_Date_Time', index = range(308160))

In [77]:
df_year.shape

(308160, 1)

In [78]:
df.shape

(45270, 141)

In [80]:
df_year.shape[0]//df.shape[0]

6

In [84]:
df_year.columns = ['Y_Date_Time']

In [ ]:
df.

In [88]:
df_y_1 = pd.concat([df_year.iloc[:45270,], df], axis = 1)

In [89]:
45270*2

90540

In [90]:
df_y_2 = pd.concat([df_year.iloc[45270:90540,], df], axis = 1)

In [91]:
90540 + 45270

135810

In [92]:
df_y_2 = pd.concat([df_year.iloc[90540:135810,], df], axis = 1)

In [93]:
135810 + 45270

181080

In [94]:
df_y_3 = pd.concat([df_year.iloc[135810:181080,], df], axis = 1)

In [95]:
df_y_1.append(df_y_2).shape

(135810, 142)

In [114]:
df_y_3.shape

(90540, 142)

In [115]:
df_y_3.isnull().sum().sum()

6428340

In [116]:
pieces = {'t1': df, 't2': df, 't3': df, 't4': df, 't5': df, 't6': df}

result = pd.concat(pieces)

In [117]:
result.shape

(271620, 141)

In [118]:
df_year.shape

(308160, 1)

In [119]:
df_year.columns

Index(['Y_Date_Time'], dtype='object')

In [122]:
df_year['Y_Date_Time'][:271620].shape

(271620,)

In [123]:
result.index = df_year['Y_Date_Time'][:271620]

In [126]:
# result.to_csv('/home/ubuntu/Zabbix/Data/DEMO_DATA/anomaly_demo.csv')

In [133]:
result.reset_index(inplace=True)

In [134]:
result.columns

Index(['Y_Date_Time', 'Date_Time', 'Total disk space on K:',
       'Total disk space on J:', 'Total disk space on N:\LOGS',
       'Total disk space on N:',
       'SQL Server, Locks: Lock Waits per second (Total)',
       'SQL Server, Buffer Manager: Page reads/sec',
       'Outgoing network traffic on vmxnet3 Ethernet Adapter-WFP Native MAC Layer LightWeight Filter-0000',
       'SQL Server, Locks: Number of Deadlocks/sec (Total)',
       ...
       'Free disk space on H:\LOGS', 'Free disk space on Q:',
       'SQL Server, Memory Manager: Memory Grants Pending',
       'Free disk space on S:', 'SQL Server, Cache: Cache Hit Ratio',
       'Anomaly', 'Anomaly_Score', 'Anomaly_Feature_1_1',
       'Anomaly_Feature_2_2', 'Anomaly_Feature_3_3'],
      dtype='object', length=142)

In [176]:
new_col = ['Date_Time',
'Total_disk_space_on_K:',
"'Total_disk_space_on_J:'",
"'Total_disk_space_on_N:\LOGS'",
"'Total_disk_space_on_N:'",
"'SQL_Server,_Locks:_Lock_Waits_per_second_(Total)'",
"'SQL_Server,_Buffer_Manager:_Page_reads/sec'",
"'OUT_Traffic_on_WFP_Native_MAC_Layer_LightWeight_Filter0000'",
"'SQL_Server,_Locks:_Number_of_Deadlocks/sec_(Total)'",
"'Free_disk_space_on_V:'",
"'Total_disk_space_on_Q:\LOGS'",
"'SQL_Server,_Buffer_Manager:_Checkpoint_pages/sec'",
"'Free_disk_space_on_F:'",
"'Free_disk_space_on_N:'",
"'Total_disk_space_on_G:'",
"'Processor_load_(5_min_average)'",
"'IN_Traffic_on_WFP_Native_MAC_Layer_LightWeight_Filter0000'",
"'Total_disk_space_on_I:\LOGS'",
"'SQL_Server,_Statistics:_Failed_AutoParams/sec'",
"'SQL_Server,_Buffer_Manager:_Database_pages'",
"'Total_disk_space_on_H:\LOGS'",
"'Free_disk_space_on_P:'",
"'SQL_Server,_Buffer_Manager:_Page_life_expectancy(PLE)'",
"'Total_disk_space_on_F:'",
"'SQL_Server,_Errors:_Errors/sec_(DB_Offline_Errors)'", "'Processor_load_(15_min_average)'", "'SQL_Server,_Buffer_Manager:_Free_list_stalls/sec'", "'SQL_Server,_Cache:_Cache_Pages'", "'Total_disk_space_on_O:'", "'SQL_Server:_Logins/sec'", "'SQL_Server,_Statistics:_Unsafe_AutoParams/sec'", "'Free_disk_space_on_W:'", "'SQL_Server,_Buffer_Manager:_Page_lookups/sec'", "'IN_Traffic_on_WFP_802.3_MAC_Layer_LightWeight_Filter0000'", "'SQL_Server,_Access_Methods:_Work_files_created/sec'", "'Free_disk_space_on_D:'", "'Free_disk_space_on_L:\LOGS'", "'SQL_Server,_Buffer_Manager:_Readahead_pages/sec'", "'SQL_Server,_Memory_Manager:_Total_Server_Memory'", "'Total_disk_space_on_L:'", "'SQL_Server:_%_Processor_Time'", "'Free_disk_space_on_J:\LOGS'", "'Total_disk_space_on_R:\LOGS'", "'Free_disk_space_on_I:'", "'Total_disk_space_on_T:'", "'OUT_Traffic_on_WFP_802.3_MAC_Layer_LightWeight_Filter0000'", "'Total_disk_space_on_Q:'", "'SQL_Server,_Statistics:_SQL_ReCompilations/sec'", "'SQL_Server,_Statistics:_Safe_AutoParams/sec'", "'Total_disk_space_on_V:\LOGS'", "'Free_disk_space_on_S:\LOGS'", "'Free_disk_space_on_U:'", "'Total_disk_space_on_D:'", "'SQL_Server,_Statistics:_SQL_Compilations/Sec'", "'Free_disk_space_on_J:'", "'OUT_Traffic_on_Microsoft_Kernel_Debug_Network_Adapter'", "'Total_disk_space_on_L:\LOGS'", "'SQL_Server,_Access_Methods:_Forwarded_Records_per_second'", "'SQL_Server,_Buffer_Manager:_Lazy_writes/sec'", "'SQL_Server,_Access_Methods:_Index_Searches/sec'", "'Free_disk_space_on_O:'", "'Total_disk_space_on_P:'", "'SQL_Server:_Transactions_per_second_(Total)'", "'SQL_Server_working_set'", "'SQL_Server,_Statistics:_Batch_Requests/Sec'", "'SQL_Server:_Logouts/sec'", "'Free_disk_space_on_Q:\LOGS'", "'SQL_Server,_Locks:_Lock_Requests/sec_(Total)'", "'SQL_Server,_Memory_Manager:_Memory_Grants_Outstanding'", "'Total_disk_space_on_S:'", "'IN_Traffic_on_'", "'Free_disk_space_on_I:\LOGS'", "'Free_disk_space_on_O:\LOGS'", "'SQL_Server,_Memory_Manager:_Target_Server_Memory'", "'Total_disk_space_on_J:\LOGS'", "'Free_disk_space_on_V:\LOGS'", "'Total_disk_space_on_U:'", "'SQL_Server,_Latches:_Average_Latch_Wait_Time'", "'OUT_Traffic_on_'", "'Free_disk_space_on_K:\LOGS'", "'Total_disk_space_on_K:\LOGS'", "'SQL_Server,_Memory_Manager:_Granted_Workspace_Memory'", "'Total_disk_space_on_R:'", "'Free_disk_space_on_M:'", "'SQL_Server,_Statistics:_AutoParam_Attempts/sec'", "'SQL_Server,_Latches:_Total_Latch_Wait_Time'", "'Free_disk_space_on_U:\LOGS'", "'Free_disk_space_on_R:'", "'SQL_Server,_Access_Methods:_Work_tables_created/sec'", "'Free_disk_space_on_H:'", "'SQL_Server,_Errors:_Errors/sec_(User_Errors)'", "'SQL_Server:_Number_of_Blocked_Processes'", "'SQL_Server,_Access_Methods:_Full_Scans/sec'", "'Total_disk_space_on_U:\LOGS'", "'Total_disk_space_on_O:\LOGS'", "'SQL_Server,_Buffer_Manager:_%_Buffer_cache_hit_ratio'", "'Free_disk_space_on_K:'", "'Total_disk_space_on_S:\LOGS'", "'SQL_Server,_Errors:_Errors/sec_(Info_Errors)'", "'Total_disk_space_on_E:'", "'SQL_Server,_Buffer_Manager:_Page_writes/sec'", "'Total_disk_space_on_V:'", "'SQL_Server,_Cache:_Cache_Objects_in_use'", "'SQL_Server,_Cache:_Cache_Object_Counts'", "'Free_disk_space_on_E:'", "'Free_disk_space_on_P:\LOGS'", "'SQL_Server,_Buffer_Manager:_Target_pages'", "'SQL_Server,_Access_Methods:_Page_Splits/sec'", "'SQL_Server,_Latches:_Latch_Waits/sec'", "'Total_disk_space_on_W:'", "'Total_disk_space_on_I:'", "'SQL_Server,_Errors:_Errors/sec_(Total)'", "'Free_disk_space_on_C:'", "'IN_Traffic_on_Microsoft_Kernel_Debug_Network_Adapter'", "'SQL_Server_pf'", "'Total_disk_space_on_C:'", "'SQL_Server,_Memory_Manager:_Maximum_Workspace_Memory'", "'SQL_Server,_Locks:_Lock_Timeouts/sec_(Total)'", "'SQL_Server,_Locks:_Lock_Wait_Time_(Total)'", "'SQL_Server_vmsize'", "'Total_disk_space_on_M:'", "'Free_disk_space_on_T:'", "'SQL_Server,_Errors:_Errors/sec_(Kill_Connection_Errors)'", "'Total_disk_space_on_H:'", "'Free_disk_space_on_L:'", "'Free_memory'", "'Total_disk_space_on_P:\LOGS'", "'SQL_Server,_Locks:_Average_Wait_Time_(Total)'", "'Free_disk_space_on_R:\LOGS'", "'Free_disk_space_on_G:'", "'Free_disk_space_on_N:\LOGS'", "'Free_disk_space_on_H:\LOGS'", "'Free_disk_space_on_Q:'", "'SQL_Server,_Memory_Manager:_Memory_Grants_Pending'", "'Free_disk_space_on_S:'", "'SQL_Server,_Cache:_Cache_Hit_Ratio'", "'Anomaly'", "'Anomaly_Score'", "'Anomaly_Feature_1'", "'Anomaly_Feature_2'", "'Anomaly_Feature_3'",
"'Y_Date_Time'"]

In [151]:
len(new_col)

142

In [137]:
# Pushin data to Postgresql

In [ ]:
demo_year_sqln_11_anomaly_classification

In [140]:
import psycopg2
from io import StringIO

In [147]:
result.shape

(271620, 142)

In [158]:
result_col = list(result.columns[1:])

In [159]:
result_col.append("Y_Date_Time")

In [166]:
df_pg = result[result_col]

In [167]:
df_pg.columns = new_col

In [174]:
list(df_pg.columns)

["'Date_Time'",
 "'Total_disk_space_on_K:'",
 "'Total_disk_space_on_J:'",
 "'Total_disk_space_on_N:\\LOGS'",
 "'Total_disk_space_on_N:'",
 "'SQL_Server,_Locks:_Lock_Waits_per_second_(Total)'",
 "'SQL_Server,_Buffer_Manager:_Page_reads/sec'",
 "'OUT_Traffic_on_WFP_Native_MAC_Layer_LightWeight_Filter0000'",
 "'SQL_Server,_Locks:_Number_of_Deadlocks/sec_(Total)'",
 "'Free_disk_space_on_V:'",
 "'Total_disk_space_on_Q:\\LOGS'",
 "'SQL_Server,_Buffer_Manager:_Checkpoint_pages/sec'",
 "'Free_disk_space_on_F:'",
 "'Free_disk_space_on_N:'",
 "'Total_disk_space_on_G:'",
 "'Processor_load_(5_min_average)'",
 "'IN_Traffic_on_WFP_Native_MAC_Layer_LightWeight_Filter0000'",
 "'Total_disk_space_on_I:\\LOGS'",
 "'SQL_Server,_Statistics:_Failed_AutoParams/sec'",
 "'SQL_Server,_Buffer_Manager:_Database_pages'",
 "'Total_disk_space_on_H:\\LOGS'",
 "'Free_disk_space_on_P:'",
 "'SQL_Server,_Buffer_Manager:_Page_life_expectancy(PLE)'",
 "'Total_disk_space_on_F:'",
 "'SQL_Server,_Errors:_Errors/sec_(DB_Offlin

In [197]:
df_pg.dtypes

'Date_Time'                              object
'Total_disk_space_on_K:'                float64
'Total_disk_space_on_J:'                float64
'Total_disk_space_on_N:\LOGS'           float64
'Total_disk_space_on_N:'                float64
                                      ...      
'Anomaly_Score'                         float64
'Anomaly_Feature_1'                      object
'Anomaly_Feature_2'                      object
'Anomaly_Feature_3'                      object
'Y_Date_Time'                    datetime64[ns]
Length: 142, dtype: object

In [208]:
conn = psycopg2.connect(user = "test_user",
                                  password = "test_user",
                                  host = "127.0.0.1",
                                  port = "5432",
                                  database = "postgres")

# Initialize a string buffer
sio = StringIO()
sio.write(df_pg_copy.to_csv(index=True, header=True))  # Write the Pandas DataFrame as a csv to the buffer
sio.seek(0)  # Be sure to reset the position to the start of the stream

0

In [209]:
# Copy the string buffer to the database, as if it were an actual file
with conn.cursor() as c:
    c.copy_from(sio, "demo_year_sqln_11_anomaly_classification", sep=',')#, columns=list(df_pg.columns), sep=',')
    conn.commit()

BadCopyFileFormat: extra data after last expected column
CONTEXT:  COPY demo_year_sqln_11_anomaly_classification, line 1: ",'Date_Time','Total_disk_space_on_K:','Total_disk_space_on_J:','Total_disk_space_on_N:\LOGS','Total_..."


In [200]:
df_pg.head()

,'Date_Time','Total_disk_space_on_K:','Total_disk_space_on_J:','Total_disk_space_on_N:\LOGS','Total_disk_space_on_N:',"'SQL_Server,_Locks:_Lock_Waits_per_second_(Total)'","'SQL_Server,_Buffer_Manager:_Page_reads/sec'",'OUT_Traffic_on_WFP_Native_MAC_Layer_LightWeight_Filter0000',"'SQL_Server,_Locks:_Number_of_Deadlocks/sec_(Total)'",'Free_disk_space_on_V:',...,'Free_disk_space_on_Q:',"'SQL_Server,_Memory_Manager:_Memory_Grants_Pending'",'Free_disk_space_on_S:',"'SQL_Server,_Cache:_Cache_Hit_Ratio'",'Anomaly','Anomaly_Score','Anomaly_Feature_1','Anomaly_Feature_2','Anomaly_Feature_3','Y_Date_Time'
0,2020-06-16 05:17:00,8.938692e+11,9.663355e+10,5.365498e+09,2.899071e+11,0.0,0.0,1914432.0,0.0,1.661783e+10,...,3.090481e+10,0.0,1.396507e+10,81.8883,1,0.120815,SQL Server: Number of Blocked Processes,Free disk space on J:\LOGS,Free disk space on J:,2020-06-01 00:00:00
1,2020-06-16 05:18:00,8.938692e+11,9.663355e+10,5.365498e+09,2.899071e+11,0.0,0.0,1760608.0,0.0,1.661783e+10,...,3.090481e+10,0.0,1.396507e+10,87.7246,1,0.126713,Free disk space on J:\LOGS,SQL Server: Number of Blocked Processes,Free disk space on J:,2020-06-01 00:01:00
2,2020-06-16 05:19:00,8.938692e+11,9.663355e+10,5.365498e+09,2.899071e+11,0.0,0.0,1820848.0,0.0,1.661783e+10,...,3.090481e+10,0.0,1.396507e+10,81.2514,1,0.126454,Free disk space on J:\LOGS,SQL Server: Number of Blocked Processes,"SQL Server, Cache: Cache Hit Ratio",2020-06-01 00:02:00
3,2020-06-16 05:20:00,8.938692e+11,9.663355e+10,5.365498e+09,2.899071e+11,0.0,0.0,1720624.0,0.0,1.661783e+10,...,3.090481e+10,0.0,1.396507e+10,86.2335,1,0.125662,Free disk space on J:\LOGS,SQL Server: Number of Blocked Processes,"SQL Server, Memory Manager: Maximum Workspac",2020-06-01 00:03:00
4,2020-06-16 05:21:00,8.938692e+11,9.663355e+10,5.365498e+09,2.899071e+11,0.0,0.0,2609184.0,0.0,1.661783e+10,...,3.090481e+10,0.0,1.396507e+10,84.1061,1,0.130407,Free disk space on J:\LOGS,SQL Server: Number of Blocked Processes,Free disk space on J:,2020-06-01 00:04:00


In [212]:
[x for x in df.columns if 'Buffer' in x]

['SQL Server, Buffer Manager: Page reads/sec',
 'SQL Server, Buffer Manager: Checkpoint pages/sec',
 'SQL Server, Buffer Manager: Database pages',
 'SQL Server, Buffer Manager: Page life expectancy(PLE)',
 'SQL Server, Buffer Manager: Free list stalls/sec',
 'SQL Server, Buffer Manager: Page lookups/sec',
 'SQL Server, Buffer Manager: Read-ahead pages/sec',
 'SQL Server, Buffer Manager: Lazy writes/sec',
 'SQL Server, Buffer Manager: % Buffer cache hit ratio',
 'SQL Server, Buffer Manager: Page writes/sec',
 'SQL Server, Buffer Manager: Target pages']

In [203]:
df_pg_copy = df_pg.copy()

In [204]:
df_pg_copy.head()

,'Date_Time','Total_disk_space_on_K:','Total_disk_space_on_J:','Total_disk_space_on_N:\LOGS','Total_disk_space_on_N:',"'SQL_Server,_Locks:_Lock_Waits_per_second_(Total)'","'SQL_Server,_Buffer_Manager:_Page_reads/sec'",'OUT_Traffic_on_WFP_Native_MAC_Layer_LightWeight_Filter0000',"'SQL_Server,_Locks:_Number_of_Deadlocks/sec_(Total)'",'Free_disk_space_on_V:',...,'Free_disk_space_on_Q:',"'SQL_Server,_Memory_Manager:_Memory_Grants_Pending'",'Free_disk_space_on_S:',"'SQL_Server,_Cache:_Cache_Hit_Ratio'",'Anomaly','Anomaly_Score','Anomaly_Feature_1','Anomaly_Feature_2','Anomaly_Feature_3','Y_Date_Time'
0,2020-06-16 05:17:00,8.938692e+11,9.663355e+10,5.365498e+09,2.899071e+11,0.0,0.0,1914432.0,0.0,1.661783e+10,...,3.090481e+10,0.0,1.396507e+10,81.8883,1,0.120815,SQL Server: Number of Blocked Processes,Free disk space on J:\LOGS,Free disk space on J:,2020-06-01 00:00:00
1,2020-06-16 05:18:00,8.938692e+11,9.663355e+10,5.365498e+09,2.899071e+11,0.0,0.0,1760608.0,0.0,1.661783e+10,...,3.090481e+10,0.0,1.396507e+10,87.7246,1,0.126713,Free disk space on J:\LOGS,SQL Server: Number of Blocked Processes,Free disk space on J:,2020-06-01 00:01:00
2,2020-06-16 05:19:00,8.938692e+11,9.663355e+10,5.365498e+09,2.899071e+11,0.0,0.0,1820848.0,0.0,1.661783e+10,...,3.090481e+10,0.0,1.396507e+10,81.2514,1,0.126454,Free disk space on J:\LOGS,SQL Server: Number of Blocked Processes,"SQL Server, Cache: Cache Hit Ratio",2020-06-01 00:02:00
3,2020-06-16 05:20:00,8.938692e+11,9.663355e+10,5.365498e+09,2.899071e+11,0.0,0.0,1720624.0,0.0,1.661783e+10,...,3.090481e+10,0.0,1.396507e+10,86.2335,1,0.125662,Free disk space on J:\LOGS,SQL Server: Number of Blocked Processes,"SQL Server, Memory Manager: Maximum Workspac",2020-06-01 00:03:00
4,2020-06-16 05:21:00,8.938692e+11,9.663355e+10,5.365498e+09,2.899071e+11,0.0,0.0,2609184.0,0.0,1.661783e+10,...,3.090481e+10,0.0,1.396507e+10,84.1061,1,0.130407,Free disk space on J:\LOGS,SQL Server: Number of Blocked Processes,Free disk space on J:,2020-06-01 00:04:00


In [234]:
len(df[df['Anomaly'] == -1]['Anomaly_Feature_1_1'].unique())

17

In [207]:
# df_pg_copy.to_csv('/home/ubuntu/Zabbix/Data/DEMO_DATA/anomaly_demo_df_pg.csv')